In [385]:
# pad your sequences

from torch.nn.utils.rnn import pad_sequence
import torch
import numpy as np
import os
import json
import joblib
from torch.utils.data import Dataset,DataLoader
from itertools import repeat
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger



In [386]:
# target = torch.randn(3,4)
# query = torch.randn(1,4)



# out = pad_sequence([target,query],batch_first=True)
# out.shape
import numpy as np

In [387]:
# get a list of data in your dataloader
# len(data) is not divisible by batch_size on purpose to verify consistency across batch sizes
#data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,\
#  collate_fn=lambda x: x)

In [388]:

# batch = [torch.rand(23,4),torch.rand(10,4),torch.rand(2,4)]

# sq_lens = list(map(lambda x:x.size(0),batch))
# ln_key = batch[0].size(-1)
# # mask = (batch_size, 1, length_key)(all queries have same mask)
# mask = torch.ones(len(sq_lens),1,max(sq_lens))

# for ind,ele in enumerate(sq_lens):
#     mask[ind,:,:ele] = 0.0



In [389]:
# #sample masking

# target = torch.randn(2,3,4)
# query = torch.randn(2,1,4)
# mask = torch.tensor(np.stack([np.array([[True,True,False]]),np.array([[True,False,False]])]))

# attn_dec_layer = MultiHeadAttention(4, 1, dropout_rate=0) 






## Making the dataset of balanced size

In [390]:
import torch
import numpy as np
import os
import json
import joblib
from torch.utils.data import Dataset,DataLoader
from itertools import repeat
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger

UTILS

In [391]:
#utils

def get_vids(base_dir,split):
    trn_split = base_dir+split
    trn_idlst = []
    trn_vidlst = []

    f = open(trn_split,'r')
    for line in f:
        id_,vid = line.split('/')
        vid = vid.strip('\n')
        trn_idlst.append(id_)
        trn_vidlst.append(vid)
        #print(vid)
        #break
    f.close()
    return trn_idlst,trn_vidlst

    
def get_features(data_dir,split='val',feat_dir='/common/users/vk405/feat_csv/'):
    #feat_dir = data_dir
    splits_dir = data_dir+'splits/'
    if split == 'val':
        feat_split_dir = feat_dir+'val_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'val_list.txt')  
    elif split == 'train':
        feat_split_dir = feat_dir+'train_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'train_list.txt') 
    elif split == 'test':
        feat_split_dir = feat_dir+'test_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'test_list.txt')
    else:
        raise NotImplementedError(f'unknown split: {split}')     
    feat_list = {}
    vid_dtls = []
    for num,name in zip(vid_num,vid_name):
        feat_loc = os.path.join(feat_split_dir, f'{num}/{name}/0001/')
        #import pdb;pdb.set_trace()
        if os.path.isdir(feat_loc):
            feat_files = feat_loc + os.listdir(feat_loc)[0]
            feat_list[name] = feat_files
            #feat_list.append(feat_files)
            vid_dtls.append((num,name))
        else:
            print(f"video : {num}/{name} not found")
    assert len(feat_list) == len(vid_dtls),"get-features is giving incorrect features"
    return feat_list,vid_dtls






def get_raw_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns['database']:
                #import pdb;pdb.set_trace()
                duration = annotns['database'][vidname]['duration']
                annot = annotns['database'][vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    sent = segment_info['sentence']
                    labels.append((interval,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

def regress_labels(raw_labels):
    regress_labels = {}
    for key in raw_labels:
        new_labels = []
        for item in raw_labels[key]:
            rng,sent,vidlen = item
            mid = sum(rng)/2
            duration = rng[-1]-rng[0]
            mid_pred = (1/vidlen)*mid # location of mid-point w.r.t video length
            duration_pred = (1/vidlen)*duration
            new_labels.append(([mid_pred,duration_pred],sent))
        regress_labels[key] = new_labels
    return regress_labels
            
            
    
    

def get_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        #print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns:
                #import pdb;pdb.set_trace()
                duration = annotns[vidname]['duration']
                annot = annotns[vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    st_end = [interval[0],interval[-1]]
                    sent = segment_info['sentence']
                    labels.append((st_end,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info
    


In [392]:


# #dataset
# # Dataset/loader
# # This is newer version
# class YoucookDset2(Dataset):
#     def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
#         ,split='train',use_precomp_emb=True,seqlen=26,framecnt=499,id=0):
#         self.id = id
#         self.feat_locs = {}
#         self.split = split
#         self.data_dir = data_dir
#         self.use_precomp_emb = use_precomp_emb
#         self.text_emb = None
#         self.seqlen = seqlen
#         self.framecnt = framecnt
#         if self.split != 'test':
#             self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
#         else:
#             raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
#         if self.use_precomp_emb:
#             self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))
#         #feat_locs = {'Ysh60eirChU': location of the video}
#         self.feat_locs,vids = get_features(self.data_dir,split=self.split)
#         assert len(vids) == len(self.feat_locs),"features are wrong"
#         #import pdb;pdb.set_trace()
#         label_info = get_labels(vids,self.annotns_file)
#         #self.labelencoder = LabelEncoder2()
#         self.final_labels = label_info
#         #self.labelencoder.fit_transform(label_info)
        
#         #regress_labels(label_info)
#         #(vid_id,seg_id)
#         self.update_data()

                
            
#     def __len__(self):
#         return len(self.data)

#     def update_data(self,id=None):
#         self.data = []
#         #self.vid_len = []
#         if not id:
#             id = self.id
            
#         starting_pnt = np.arange(id,self.framecnt,self.seqlen)

#         for key in self.final_labels:
#             annot_len = len(self.final_labels[key])
#             if key in self.feat_locs:
#                 file_loc = self.feat_locs[key]
#                 #for stpnt in starting_pnt:
#                 segments = list(zip(repeat(key,annot_len),repeat(file_loc,annot_len),\
#                         range(annot_len)))
#                 for seg in segments:
#                     for stpnt in starting_pnt:
#                         if stpnt+self.seqlen<=self.framecnt:
#                             datapnt = seg[:-1]+(stpnt,)+seg[-1:]
#                             self.data.append(datapnt)
                    
#                 #self.data.extend(segments)
#             else:
#                 print(f"video:{key} not found")


#     def getclass_prob(self,lbl_rng,frame_rng):
#         lbl_ids = set(np.arange(lbl_rng[0],lbl_rng[-1]+1))
#         frame_ids = set(np.arange(frame_rng[0],frame_rng[-1]+1))
#         inter = lbl_ids.intersection(frame_ids)
#         if len(inter) == 0:
#             return 0.0
#         else:
#             return len(inter)/len(lbl_ids.union(frame_ids))

#     def __getitem__(self,idx):
#         if self.use_precomp_emb:
#             vidname,file_loc,stid,seg_ind = self.data[idx]
#             #import pdb;pdb.set_trace()
#             #self.txt_emb[vidname][seg_ind],
#             txt_info = self.final_labels[vidname][seg_ind]
#             label_value = self.getclass_prob(txt_info[0],(stid,stid+self.seqlen-1))
#             #import pdb;pdb.set_trace()
#             return pd.read_csv(file_loc).values.astype(np.float32)[stid:stid+self.seqlen,:],(self.txt_emb[vidname][seg_ind]).astype(np.float32),\
#                 label_value
#             #np.array(self.final_labels[vidname][seg_ind][0],dtype=np.float32)
#         else:
#             raise NotImplementedError("not yet correctly implemented")

        

           



        

In [393]:
# youcookdata = YoucookDset2()

In [394]:
# youcookdata.final_labels['Ysh60eirChU']

In [395]:
# youcookdata.data[0]

In [396]:
# youcookdata.feat_locs['Ysh60eirChU']

In [397]:
# from collections import namedtuple
# #Point = namedtuple("vid_id", "vid_loc","start","end","segid","label")

# def overlap_frac(base_rng,tst_rng):
#     #1.Returns the fraction of frames that are overlapping in tst_rng with base_rng
#     #2.both ends inclusive
#     sz = tst_rng[-1]-tst_rng[0]+1
#     lbl_ids = set(np.arange(base_rng[0],base_rng[-1]+1))
#     frame_ids = set(np.arange(tst_rng[0],tst_rng[-1]+1))
#     inter = frame_ids.intersection(lbl_ids)
#     assert sz != 0,"base frame rng is zero"
#     return len(inter)/sz
    

# data = []
# max_cnt = 50
# for key in youcookdata.final_labels:
#     segments = youcookdata.final_labels[key]
#     for ind,seg in enumerate(segments):
#         #trn_points = []
#         st_end,txt,vid_len = seg
#         main_seg = (key,youcookdata.feat_locs[key],st_end[0],st_end[-1],ind,1.0)
#         data.append(main_seg)
#         frame_width = st_end[-1]-st_end[0] + 1
#         extra_frames = []
#         for cnt,new_st in enumerate(range(st_end[0]+1,st_end[-1]+1)):
#             #forward sliding
#             new_end = new_st+frame_width
#             if (cnt<max_cnt)and (0<=new_st<youcookdata.framecnt and 0<=new_st<youcookdata.framecnt):
#                 extra_frames.append((new_st,new_end))
#         for cnt,new_end in enumerate(range(st_end[-1],st_end[0],-1)):
#             #backward sliding
#             new_st = new_end-frame_width
#             if (cnt<max_cnt)and (0<=new_st<youcookdata.framecnt and 0<=new_st<youcookdata.framecnt):
#                 extra_frames.append((new_st,new_end))
#         #import pdb;pdb.set_trace()
#         for ex_seg in extra_frames:
#             label = overlap_frac(st_end,ex_seg)
#             data.append((key,youcookdata.feat_locs[key],ex_seg[0],ex_seg[-1],ind,label))


        

        

        
        

    

In [398]:
# all values are equally distributed
#df


#utils



def get_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        #print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns:
                #import pdb;pdb.set_trace()
                duration = annotns[vidname]['duration']
                annot = annotns[vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    st_end = [interval[0],interval[-1]]
                    sent = segment_info['sentence']
                    labels.append((st_end,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

def get_vids(base_dir,split):
    trn_split = base_dir+split
    trn_idlst = []
    trn_vidlst = []

    f = open(trn_split,'r')
    for line in f:
        id_,vid = line.split('/')
        vid = vid.strip('\n')
        trn_idlst.append(id_)
        trn_vidlst.append(vid)
        #print(vid)
        #break
    f.close()
    return trn_idlst,trn_vidlst

    
def get_features(data_dir,split='val',feat_dir='/common/users/vk405/feat_csv/'):
    #feat_dir = data_dir
    splits_dir = data_dir+'splits/'
    if split == 'val':
        feat_split_dir = feat_dir+'val_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'val_list.txt')  
    elif split == 'train':
        feat_split_dir = feat_dir+'train_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'train_list.txt') 
    elif split == 'test':
        feat_split_dir = feat_dir+'test_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'test_list.txt')
    else:
        raise NotImplementedError(f'unknown split: {split}')     
    feat_list = {}
    vid_dtls = []
    for num,name in zip(vid_num,vid_name):
        feat_loc = os.path.join(feat_split_dir, f'{num}/{name}/0001/')
        #import pdb;pdb.set_trace()
        if os.path.isdir(feat_loc):
            feat_files = feat_loc + os.listdir(feat_loc)[0]
            feat_list[name] = feat_files
            #feat_list.append(feat_files)
            vid_dtls.append((num,name))
        else:
            print(f"video : {num}/{name} not found")
    assert len(feat_list) == len(vid_dtls),"get-features is giving incorrect features"
    return feat_list,vid_dtls






def get_raw_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns['database']:
                #import pdb;pdb.set_trace()
                duration = annotns['database'][vidname]['duration']
                annot = annotns['database'][vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    sent = segment_info['sentence']
                    labels.append((interval,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

def regress_labels(raw_labels):
    regress_labels = {}
    for key in raw_labels:
        new_labels = []
        for item in raw_labels[key]:
            rng,sent,vidlen = item
            mid = sum(rng)/2
            duration = rng[-1]-rng[0]
            mid_pred = (1/vidlen)*mid # location of mid-point w.r.t video length
            duration_pred = (1/vidlen)*duration
            new_labels.append(([mid_pred,duration_pred],sent))
        regress_labels[key] = new_labels
    return regress_labels
            
            
    
    
    


In [399]:


#dataset
# Dataset/loader
# This is newer version
class YoucookDset2(Dataset):
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',framecnt=499):
        self.feat_locs = {}
        self.split = split
        self.data_dir = data_dir
        self.framecnt = framecnt
        #self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        # if self.use_precomp_emb:
        #     self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))
        #feat_locs = {'Ysh60eirChU': location of the video}
        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        assert len(vids) == len(self.feat_locs),"features are wrong"
        #import pdb;pdb.set_trace()
        #label_info = get_labels(vids,self.annotns_file)
        #self.labelencoder = LabelEncoder2()
        self.final_labels = get_labels(vids,self.annotns_file)
        #self.labelencoder.fit_transform(label_info)
        
        #regress_labels(label_info)
        #(vid_id,seg_id)
        self.data = self.update_data()

                
            
    def __len__(self):
        return len(self.data)

    def overlap_frac(self,base_rng,tst_rng):
        #1.Returns the fraction of frames that are overlapping in tst_rng with base_rng
        #2.both ends inclusive
        sz = tst_rng[-1]-tst_rng[0]+1
        lbl_ids = set(np.arange(base_rng[0],base_rng[-1]+1))
        frame_ids = set(np.arange(tst_rng[0],tst_rng[-1]+1))
        inter = frame_ids.intersection(lbl_ids)
        assert sz != 0,"base frame rng is zero"
        return len(inter)/sz


    def update_data(self):
        data = []
        max_cnt = 50
        for key in self.final_labels:
            segments = self.final_labels[key]
            for ind,seg in enumerate(segments):
                #trn_points = []
                st_end,txt,vid_len = seg
                main_seg = (key,self.feat_locs[key],st_end[0],st_end[-1],ind,1.0)
                data.append(main_seg)
                frame_width = st_end[-1]-st_end[0] + 1
                extra_frames = []
                for cnt,new_st in enumerate(range(st_end[0]+1,st_end[-1]+1)):
                    #forward sliding
                    new_end = new_st+frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                for cnt,new_end in enumerate(range(st_end[-1],st_end[0],-1)):
                    #backward sliding
                    new_st = new_end-frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                #import pdb;pdb.set_trace()
                for ex_seg in extra_frames:
                    label = self.overlap_frac(st_end,ex_seg)
                    data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,label))
        return data

    def __getitem__(self,idx):
        return self.data[idx]
        

           



        

In [400]:
ydata = YoucookDset2()

In [401]:
# from torch.utils.data import Dataset,TensorDataset
# inps = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
# tgts = torch.arange(10 * 5, dtype=torch.float32).view(10, 5)
# dataset = TensorDataset(inps, tgts)

In [402]:
# class ToyDataset(Dataset):
#     def __init__(self):
#         self.data = ['h','l','e','o']
#     def __len__(self):
#         return len(self.data)
#     def __getitem__(self,idx):
#         return self.data[idx]

# dataset = ToyDataset()

In [403]:
# len(dataset)

In [404]:
from collections import defaultdict
# data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'
# global_txt = joblib.load(os.path.join(data_dir,'emb.joblib'))
def collate_wrapper(data):
    labels = []
    vid_embs = []
    txt_embs = []
    #namedtuple("vid_id", "vid_loc","start","end","segid","label")
    batched_data = pd.DataFrame(data,columns=["vid_id", "vid_loc","start","end","segid","label"])
    unique_locs = batched_data['vid_loc'].unique()
    for loc in unique_locs:
        locwise = batched_data[batched_data['vid_loc']==loc]
        tot_vid = pd.read_csv(loc).values
        txtemb = None
        for ind,ele in locwise.iterrows():
            vid_id,_,st,end,segid,label = ele
            #import pdb;pdb.set_trace()
            if not txtemb:
                txtemb = global_txt[ele['vid_id']]
            vid_embs.append(torch.tensor(tot_vid[ele['start']:ele['end']+1]))
            txt_embs.append(torch.tensor(txtemb[ele['segid']]))
            labels.append(torch.tensor(ele['label']))
    #return (np.stack(vid_embs),np.stack(txt_embs)),np.stack(labels)
    return vid_embs,torch.stack(txt_embs),torch.stack(labels)
        
   

In [405]:
# # get a dataloader
# ydata = YoucookDset2()
# dl_new = DataLoader(ydata,batch_size=1000,shuffle=True,collate_fn=collate_wrapper)
# batch = next(iter(dl_new))

In [406]:
#batch[0][1].shape

Model utils 

In [407]:
# Now changing the model to account for this change


#model utils

#!pip install transformers

def init_parameters_xavier_uniform(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def scaled_dot(query, key, mask_key=None):  
    score = torch.matmul(query, key.transpose(-2, -1))
    score /= math.sqrt(query.size(-1))
    if mask_key is not None:
        score = score.masked_fill(mask_key, -1e18)  # Represents negative infinity
    return score      
            
def attend(query, key, value, mask_key=None, dropout=None):
    # TODO: Implement
    # Use scaled_dot, be sure to mask key
    #smax = nn.Softmax(-1)
    #import pdb;pdb.set_trace()
    score = scaled_dot(query,key,mask_key)  
    attention = F.softmax(score,dim=-1)
    if dropout is not None:#do = nn.Dropout(dropout)
        attention = dropout(attention)
    answer = torch.matmul(attention,value) 
    # Convexly combine value embeddings using attention, this should be just a matrix-matrix multiplication.
    return answer, attention



def split_heads(batch, num_heads):  
    (batch_size, length, dim) = batch.size()  # These are the expected batch dimensions.
    assert dim % num_heads == 0  # Assert that dimension is divisible by the number of heads.
    dim_head = dim // num_heads

    # No new memory allocation
    splitted = batch.view(batch_size, -1, num_heads, dim_head).transpose(1, 2)  
    return splitted  # (batch_size, num_heads, length, dim_head), note that now the last two dimensions are compatible with our attention functions. 




def merge_heads(batch):  
    (batch_size, num_heads, length, dim_head) = batch.size()  # These are the expected batch dimensions.

    # New memory allocation (reshape), can't avoid.
    merged = batch.transpose(1, 2).reshape(batch_size, -1, num_heads * dim_head)
    return merged  # (batch_size, length, dim)


class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads, dropout_rate=0.1):
        super().__init__()
        assert dim % num_heads == 0

        self.linear_query = nn.Linear(dim, dim)
        self.linear_key = nn.Linear(dim, dim)
        self.linear_value = nn.Linear(dim, dim)
        self.linear_final = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout_rate)

        self.num_heads = num_heads

    def forward(self, query, key, value, mask_key=None, layer_cache=None,
              memory_attention=False):
        """
        INPUT
          query: (batch_size, length_query, dim)
          key: (batch_size, length_key, dim)
          value: (batch_size, length_key, dim_value)
          mask_key: (*, 1, length_key) if queries share the same mask, else
                    (*, length_query, length_key)
          layer_cache: if not None, stepwise decoding (cache of key/value)
          memory_attention: doing memory attention in stepwise decoding?
        OUTPUT
          answer: (batch_size, length_query, dim_value)
          attention: (batch_size, num_heads, length_query, length_key) else
        """
        batch_size = query.size(0)

        query = self.linear_query(query)
        query = split_heads(query, self.num_heads)  # (batch_size, num_heads, -1, dim_head)

        def process_key_value(key, value):  # Only called when necessary.
            key = self.linear_key(key)
            key = split_heads(key, self.num_heads)
            value = self.linear_value(value)
            value = split_heads(value, self.num_heads)
            return key, value

        #import pdb;pdb.set_trace()
        if layer_cache is None:
            key, value = process_key_value(key, value)
        else:
            assert query.size(2) == 1  # Stepwise decoding
            
            if memory_attention:
                if layer_cache['memory_key'] is None:  # One-time calculation
                    key, value = process_key_value(key, value)
                    # (batch_size, num_heads, length_memory, dim)
                    layer_cache['memory_key'] = key
                    layer_cache['memory_value'] = value

                key = layer_cache['memory_key']
                value = layer_cache['memory_value']

            else:  # Self-attention during decoding
                key, value = process_key_value(key, value)
                assert key.size(2) == 1 and value.size(2) == 1
                
                # Append to previous.
                if layer_cache['self_key'] is not None:
                    key = torch.cat((layer_cache['self_key'], key), dim=2)
                    value = torch.cat((layer_cache['self_value'], value), dim=2)
                    
                 # (batch_size, num_heads, length_decoded, dim)
                layer_cache['self_key'] = key  # Recache.
                layer_cache['self_value'] = value
        # Because we've splitted embeddings into heads, we must also split the mask. 
        # And because each query uses the same mask for all heads (we don't use different masking for different heads), 
        # we can specify length 1 for the head dimension.
        if mask_key is not None:  
            mask_key = mask_key.unsqueeze(1)  # (batch_size, 1, -1, length_key)

        answer, attention = attend(query, key, value, mask_key, self.dropout)

        answer = merge_heads(answer)  # (batch_size, length_key, dim)
        answer = self.linear_final(answer)

        return answer, attention

class PositionwiseFeedForward(nn.Module):
    def __init__(self, dim, dim_hidden, drop_rate=0.1):
        super().__init__()
        self.w1 = nn.Linear(dim, dim_hidden)
        self.w2 = nn.Linear(dim_hidden, dim)
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.drop1 = nn.Dropout(drop_rate)
        self.relu = nn.ReLU()
        self.drop2 = nn.Dropout(drop_rate)
    def forward(self, x):
        inter = self.drop1(self.relu(self.w1(self.layer_norm(x))))
        output = self.drop2(self.w2(inter))
        return output + x




class SinusoidalPositioner(nn.Module):
    def __init__(self, dim, drop_rate=0.1, length_max=5000):
        super().__init__()
        frequency = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.) / dim))  # Using different frequency for each dim
        positions = torch.arange(0, length_max).unsqueeze(1)
        wave = torch.zeros(length_max, dim)
        wave[:, 0::2] = torch.sin(frequency * positions)
        wave[:, 1::2] = torch.cos(frequency * positions)
        self.register_buffer('wave', wave.unsqueeze(0))  # (1, length_max, dim)
        self.dropout = nn.Dropout(drop_rate)
        self.dim = dim
        self.length_max = length_max
    def forward(self, x, step=-1):
        assert x.size(-2) <= self.length_max

        if step < 0:  # Take the corresponding leftmost embeddings.
            position_encoding = self.wave[:, :x.size(-2), :]
        else:  # Take the embedding at the step.
            position_encoding = self.wave[:, step, :]

        x = x * math.sqrt(self.dim)
        return self.dropout(x + position_encoding)





class CrossAttentionLayer(nn.Module):
    def __init__(self,dim,num_heads,dim_hidden,drop_rate):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.context_attention = MultiHeadAttention(dim, num_heads, drop_rate)
        self.drop = nn.Dropout(drop_rate)
        self.feedforward = PositionwiseFeedForward(dim, dim_hidden, drop_rate)
        
    def forward(self,target,memory,layer_cache=None,mask_key=None):
        
        cross_attn_target = self.layer_norm(target)
        attended, attention = self.context_attention(cross_attn_target,memory,memory,layer_cache=layer_cache,memory_attention=True,mask_key=mask_key)
        
        attended = target + self.drop(attended)
        
        return self.feedforward(attended),attention



layer_cache = {'memory_key': None, 'memory_value': None, 'self_key': None, 'self_value': None}

Model Main


In [450]:
#model 






class CrossattnModel(pl.LightningModule):
    def __init__(self,hparams,dset=None):
        
        super().__init__()
        self.save_hyperparameters(hparams)
        #self.hparams = hparams
        #import pdb;pdb.set_trace()
        #self.net= Model(hparams)
        #self.hparams  = hparams
        
        self.positioner = SinusoidalPositioner(self.hparams.edim, drop_rate=0., length_max=1000)
        self.attn = CrossAttentionLayer(self.hparams.edim,self.hparams.nheads,\
                           self.hparams.attnhdim,self.hparams.dropoutp)
        self.wrdcnn =  nn.Conv1d(self.hparams.wrdim, self.hparams.edim, 1, stride=1)
        self.vidcnn =  nn.Conv1d(self.hparams.vidim, self.hparams.edim, 1, stride=1)
        self.hid_layer = nn.Linear(self.hparams.edim,self.hparams.hdim)
        self.out_layer = nn.Linear(self.hparams.hdim,1)
        #self
        self.init_parameters_xavier_uniform()
        self.dset = dset

    def forward(self,x):
        #keep this for inference
        vid_feat,wrd_feat = x
        mask = self.get_mask(vid_feat)
        # pad the features with zeros
        vid_feat = pad_sequence(vid_feat,batch_first=True)
        out = self.net((vid_feat.float(),wrd_feat.float()),mask_key=mask)
        return out
        
    def net(self,x,mask_key=None):
        vid_x,wrd_x = x
        
        wrd_x = wrd_x.unsqueeze(1).transpose(1,2)
        vid_x = vid_x.transpose(1,2)
        #print(f"inside model, wrd_x:{wrd_x.shape},vi")
        tgt = self.wrdcnn(wrd_x.float()).transpose(1,2)
        src = self.vidcnn(vid_x.float()).transpose(1,2)
        src_posencode = self.positioner(src)
        #for i in range(self.hparams.lyrs):
        # ?create mask_key and send it
        attended,attn_score = self.attn(tgt,src_posencode,mask_key=mask_key)
            #tgt = 
        out = F.sigmoid(self.out_layer(F.relu(self.hid_layer(F.relu(attended)))))
        return out

    def get_mask(self,batch):
        #device = batch.device
        sq_lens = list(map(lambda x:x.size(0),batch))
        ln_key = batch[0].size(-1)
        # mask = (batch_size, 1, length_key)(all queries have same mask)
        mask = torch.ones(len(sq_lens),1,max(sq_lens))
        for ind,ele in enumerate(sq_lens):
            mask[ind,:,:ele] = 0.0
        return mask


    def training_step(self,batch,batch_idx):
        #for tranining
        vid_feat,wrd_feat,labels = batch
        # here vid_feat is list of video frames of varying length
        mask = self.get_mask(vid_feat)
        # pad the features with zeros
        vid_feat = pad_sequence(vid_feat,batch_first=True)
        x_hat = self.net((vid_feat.float(),wrd_feat.float()),mask_key=mask)
        if x_hat.shape[0] != self.hparams.batch_size:
            print(f"*****batch size mismatch: running bs:{x_hat.shape[0]}*****")
            #import pdb;pdb.set_trace()
        #loss = nn.BCELoss()
        loss = F.binary_cross_entropy(x_hat.squeeze().float(), labels.squeeze().float())
        #print(f"inside train step, loss:{loss}")
        self.log("train_loss",loss,on_step=True)
        return loss

    def validation_step(self,batch,batch_idx):
        #for validation
        vid_feat,wrd_feat,labels = batch
        mask = self.get_mask(vid_feat)
        vid_feat = pad_sequence(vid_feat,batch_first=True)
        x_hat = self.net((vid_feat.float(),wrd_feat.float()),mask_key=mask)
        
        loss = F.binary_cross_entropy(x_hat.squeeze().float(), labels.squeeze().float())
        #print(f"inside train step, loss:{loss}")
        self.log("val_loss",loss,on_step=False, on_epoch=True)
        return loss


    def configure_optimizers(self):
        #lr = self.hparams.lr if 'lr' in self.hparams else 1e-3
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)

        # Apply lr scheduler per step
        lr_scheduler = CosineWarmupScheduler(optimizer,
                                             warmup=self.hparams.warmup,
                                             max_iters=self.hparams.max_iters)
        return [optimizer], [{'scheduler': lr_scheduler, 'interval': 'step'}]

    
    def init_parameters_xavier_uniform(self):
        for name,p in self.named_parameters():
            if 'attn' in name and p.dim() > 1:
                nn.init.xavier_uniform_(p)

    
        



In [449]:
youcookdata = YoucookDset2()
train_loader = DataLoader(youcookdata,\
            batch_size=cfg.batch_size,collate_fn=collate_w,\
                shuffle=True)

In [451]:
batch = next(iter(train_loader))

In [460]:
vid_feat,wrd_feat,labels = batch

In [461]:
vid_feat[0].shape

torch.Size([18, 512])

In [463]:
vid_feat = pad_sequence(vid_feat,batch_first=True)

In [457]:
vid_feat.shape

torch.Size([512, 223, 512])

In [459]:
wrd_feat.shape

torch.Size([512, 768])

In [462]:
conv1d = nn.Conv1d(512, 256, 2, stride=1)

In [470]:
lstm = nn.LSTM(input_size=512,hidden_size=100,num_layers=2,batch_first=True,bidirectional=False)

In [471]:
output, (hn, cn) = lstm(vid_feat.float())

In [472]:
output.shape,hn.shape,cn.shape

(torch.Size([512, 223, 100]),
 torch.Size([2, 512, 100]),
 torch.Size([2, 512, 100]))

In [ ]:
vid_feat,wrd_feat = x
        mask = self.get_mask(vid_feat)
        # pad the features with zeros
        vid_feat = pad_sequence(vid_feat,batch_first=True)
        out = self.net((vid_feat.float(),wrd_feat.float()),mask_key=mask)
        return out

In [409]:

# ydata = YoucookDset2()
# dl_new = DataLoader(ydata,batch_size=10,shuffle=True,collate_fn=collate_wrapper)

In [410]:
# input = torch.tensor([1.0,1.0])
# target = torch.tensor([0.0,0.8])
# F.binary_cross_entropy(input,target)

In [411]:
#4096 works
import wandb
import logging
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
import shutil

logger = logging.getLogger(__name__)
wandb_logger = lambda dir, version: WandbLogger(
    name="wandb", save_dir=dir, version=version
)
csvlogger = lambda dir, version: CSVLogger(dir, name="csvlogs", version=version)
tblogger = lambda dir, version: TensorBoardLogger(dir, name="tblogs", version=version)

def get_loggers(dir,version,lis=["csv"]):
    lgrs = []
    if "wandb" in lis:
        lgrs.append(wandb_logger(dir, version))
    if "csv" in lis:
        lgrs.append(csvlogger(dir, version))
    if "tb" in lis:
        lgrs.append(tblogger(dir, version))
    return lgrs





In [502]:


#dataset
# Dataset/loader
# This is newer version
class YoucookDset2(Dataset):
    global_txt = joblib.load(os.path.join('/common/home/vk405/Projects/Crossmdl/Data/YouCookII/','emb.joblib'))
    global_imgfeats = joblib.load('/common/users/vk405/feat_csv/global_imgfeats.joblib')
    global_map = joblib.load('/common/users/vk405/feat_csv/global_map.joblib')
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',framecnt=499):
        self.feat_locs = {}
        self.split = split
        self.data_dir = data_dir
        self.framecnt = framecnt
        #self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        # if self.use_precomp_emb:
        #     self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))
        #feat_locs = {'Ysh60eirChU': location of the video}
        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        #import pdb;pdb.set_trace()
        assert len(vids) == len(self.feat_locs),"features are wrong"
        #import pdb;pdb.set_trace()
        #label_info = get_labels(vids,self.annotns_file)
        #self.labelencoder = LabelEncoder2()
        self.final_labels = get_labels(vids,self.annotns_file)
        #self.labelencoder.fit_transform(label_info)
        
        #regress_labels(label_info)
        #(vid_id,seg_id)
        self.data = self.update_data()

                
            
    def __len__(self):
        return len(self.data)

    def overlap_frac(self,base_rng,tst_rng):
        #1.Returns the fraction of frames that are overlapping in tst_rng with base_rng
        #2.both ends inclusive
        sz = tst_rng[-1]-tst_rng[0]+1
        lbl_ids = set(np.arange(base_rng[0],base_rng[-1]+1))
        frame_ids = set(np.arange(tst_rng[0],tst_rng[-1]+1))
        inter = frame_ids.intersection(lbl_ids)
        assert sz != 0,"base frame rng is zero"
        return len(inter)/sz




    def update_data(self):
        data = []
        max_cnt = 50
        for key in self.final_labels:
            segments = self.final_labels[key]
            for ind,seg in enumerate(segments):
                #trn_points = []
                st_end,txt,vid_len = seg
                main_seg = (key,self.feat_locs[key],st_end[0],st_end[-1],ind,1.0)
                data.append(main_seg)
                frame_width = st_end[-1]-st_end[0] + 1
                extra_frames = []
                #st_end[-1]+1
                for cnt,new_st in enumerate(range(st_end[0]+1,499)):
                    #forward sliding
                    new_end = new_st+frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                #st_end[0]
                for cnt,new_end in enumerate(range(st_end[-1],0,-1)):
                    #backward sliding
                    new_st = new_end-frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                
                #import pdb;pdb.set_trace()
                zero_overlap_frames = []
                for ex_seg in extra_frames:
                    label = self.overlap_frac(st_end,ex_seg)
                    # only negative samples
                    if label == 0.00:
                        zero_overlap_frames.append(ex_seg)
                        #data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,label))
                        #break
                if len(zero_overlap_frames):
                    frame_id = np.random.randint(len(zero_overlap_frames))
                    ex_seg = zero_overlap_frames[frame_id]
                    #print(f"main_seg:{main_seg},ex_seg:{ex_seg}")
                    data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,0.0))
                    
        return data
    
    def read_data_global(self,data):
        vid_id,vid_loc,start,end,segid,label = data
        ind = self.global_map[vid_id]
        img_feat = self.global_imgfeats[ind][start:end+1]
        txt_feat = self.global_txt[vid_id][segid]
        return img_feat,txt_feat,label

    def __getitem__(self,idx):
        return self.read_data_global(self.data[idx])
        

           



        

In [413]:
#ydata = YoucookDset2()
# out = ydata[0]

In [504]:


#dataset
# Dataset/loader
# This is newer version
class YoucookDset2(Dataset):
    global_txt = joblib.load(os.path.join('/common/home/vk405/Projects/Crossmdl/Data/YouCookII/','emb.joblib'))
    global_imgfeats = joblib.load('/common/users/vk405/feat_csv/global_imgfeats.joblib')
    global_map = joblib.load('/common/users/vk405/feat_csv/global_map.joblib')
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',framecnt=499):
        self.feat_locs = {}
        self.split = split
        self.data_dir = data_dir
        self.framecnt = framecnt
        #self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        # if self.use_precomp_emb:
        #     self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))
        #feat_locs = {'Ysh60eirChU': location of the video}
        #import pdb;pdb.set_trace()
        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        #import pdb;pdb.set_trace()
        assert len(vids) == len(self.feat_locs),"features are wrong"
        #import pdb;pdb.set_trace()
        #label_info = get_labels(vids,self.annotns_file)
        #self.labelencoder = LabelEncoder2()
        self.final_labels = get_labels(vids,self.annotns_file)
        #self.labelencoder.fit_transform(label_info)
        
        #regress_labels(label_info)
        #(vid_id,seg_id)
        self.data,self.max_seq_ln= self.update_data()

                
            
    def __len__(self):
        return len(self.data)

    def overlap_frac(self,base_rng,tst_rng):
        #1.Returns the fraction of frames that are overlapping in tst_rng with base_rng
        #2.both ends inclusive
        sz = tst_rng[-1]-tst_rng[0]+1
        lbl_ids = set(np.arange(base_rng[0],base_rng[-1]+1))
        frame_ids = set(np.arange(tst_rng[0],tst_rng[-1]+1))
        inter = frame_ids.intersection(lbl_ids)
        assert sz != 0,"base frame rng is zero"
        return len(inter)/sz




    def update_data(self):
        data = []
        max_cnt = 50
        mx_seq_ln = 0
        for key in self.final_labels:
            segments = self.final_labels[key]
            for ind,seg in enumerate(segments):
                #trn_points = []
                st_end,txt,vid_len = seg
                main_seg = (key,self.feat_locs[key],st_end[0],st_end[-1],ind,1.0)
                # to keep track of max sequence length in the data
                sq_len = st_end[-1]-st_end[0]+1
                mx_seq_ln = max(mx_seq_ln,sq_len)

                data.append(main_seg)
                frame_width = st_end[-1]-st_end[0] + 1
                extra_frames = []
                #st_end[-1]+1
                for cnt,new_st in enumerate(range(st_end[0]+1,499)):
                    #forward sliding
                    new_end = new_st+frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                #st_end[0]
                for cnt,new_end in enumerate(range(st_end[-1],0,-1)):
                    #backward sliding
                    new_st = new_end-frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                
                #import pdb;pdb.set_trace()
                zero_overlap_frames = []
                for ex_seg in extra_frames:
                    label = self.overlap_frac(st_end,ex_seg)
                    # only negative samples
                    if label == 0.00:
                        zero_overlap_frames.append(ex_seg)
                        #data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,label))
                        #break
                if len(zero_overlap_frames):
                    frame_id = np.random.randint(len(zero_overlap_frames))
                    ex_seg = zero_overlap_frames[frame_id]
                    #print(f"main_seg:{main_seg},ex_seg:{ex_seg}")
                    data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,0.0))
                    sq_len = ex_seg[-1]-ex_seg[0]+1
                    mx_seq_ln = max(mx_seq_ln,sq_len)
                    
        return data,mx_seq_ln
    
    def read_data_global(self,data):
        vid_id,vid_loc,start,end,segid,label = data
        ind = self.global_map[vid_id]
        img_feat = self.global_imgfeats[ind][start:end+1]
        txt_feat = self.global_txt[vid_id][segid]
        pad_img = np.concatenate([img_feat,np.zeros((self.max_seq_ln-img_feat.shape[0],img_feat.shape[-1]))])
        return pad_img,txt_feat,len(img_feat),label

    def __getitem__(self,idx):
        return self.read_data_global(self.data[idx])
        

           



        

In [513]:
#ydata = YoucookDset2()



In [414]:
#youcookdata.final_labels

In [415]:
def collate_w(data):
    img_lis = []
    txt_lis = []
    label_lis = []
    for ele in data:
        img,txt,label = ele
        img_lis.append(torch.tensor(img))
        txt_lis.append(torch.tensor(txt))
        label_lis.append(torch.tensor(label))
    return img_lis,torch.stack(txt_lis),torch.stack(label_lis)
        
    

#train_loader = DataLoader(ydata,\
            #batch_size=100,collate_fn=collate_w)

In [416]:
# Runnin/training

data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'

class MyCollator(object):
    def __init__(self,**kwargs):
        self.kwargs = kwargs
    def __call__(self, data):
        # do something with batch and self.params
        labels = []
        vid_embs = []
        txt_embs = []
        #namedtuple("vid_id", "vid_loc","start","end","segid","label")
        batched_data = pd.DataFrame(data,columns=["vid_id", "vid_loc","start","end","segid","label"])
        unique_locs = batched_data['vid_loc'].unique()
        #import pdb;pdb.set_trace()
        for loc in unique_locs:
            locwise = batched_data[batched_data['vid_loc']==loc]
            tot_vid = pd.read_csv(loc).values
            txtemb = None
            for ind,ele in locwise.iterrows():
                #vid_id,_,st,end,segid,label = ele
                #import pdb;pdb.set_trace()
                if not txtemb:
                    txtemb = self.kwargs['txtemb'][ele['vid_id']]
                vid_embs.append(torch.tensor(tot_vid[ele['start']:ele['end']+1]))
                #import pdb;pdb.set_trace()
                txt_embs.append(torch.tensor(txtemb[ele['segid']]))
                labels.append(torch.tensor(ele['label']))
        #return (np.stack(vid_embs),np.stack(txt_embs)),np.stack(labels)
        return vid_embs,torch.stack(txt_embs),torch.stack(labels)

    



In [417]:


# from tqdm import tqdm
# from numpy import genfromtxt
# global_imgfeats = []
# global_map = {}
# i = 0
# for ele in tqdm(ydata):
#     img_id,loc,_,_,_,_ = ele
#     if img_id not in global_map:
#         global_map[img_id] = i
#         data = genfromtxt(loc,delimiter=',')
#         global_imgfeats.append(data)
#         i += 1
    
    
    


In [418]:
# import joblib
# store_p = '/common/users/vk405/feat_csv/'
# _ = joblib.dump(global_map,store_p+'global_map.joblib')
# _ = joblib.dump(global_imgfeats,store_p+'global_imgfeats.joblib')

In [419]:
# global_txt = joblib.load(os.path.join(data_dir,'emb.joblib'))

In [420]:
# global_txt = joblib.load(os.path.join(data_dir,'emb.joblib'))
# collator = MyCollator(txtemb=global_txt)
# dl_new = DataLoader(ydata,batch_size=10,shuffle=True,collate_fn=collator)
# batch = next(iter(dl_new))

In [421]:
import torch.optim as optim
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [422]:
def run(cfg):
    pl.seed_everything(cfg.seed)
    dir = cfg.artifacts_loc
    version = str(cfg.version)
    logger_list = get_loggers(dir, version,cfg.loggers)
    cbs = []
    if "early_stop" in cfg.cbs:
        #? does'nt really work atm
        params = cfg.model.cbs.early_stop
        earlystopcb = EarlyStopping(**params, min_delta=0.00, verbose=False)
        cbs.append(earlystopcb)
    if "checkpoint" in cfg.cbs:
        store_path = dir + "ckpts/" + str(cfg.version) + "/"
        isExist = os.path.exists(store_path)
        if isExist and os.path.isdir(store_path) and ('retrain' not in cfg and not cfg.retrain):
            shutil.rmtree(store_path)
        # then create fresh
        if not isExist:
            os.makedirs(store_path)
        fname = "{epoch}-{train_loss:.2f}"
        params = cfg.checkpoint
        checkptcb = ModelCheckpoint(**params, dirpath=store_path, filename=fname)
        cbs.append(checkptcb)
    if "wandb" in cfg.loggers:
        wandb.init(project="videoretrieval", config=cfg)
    if cfg.mode == 'train':
        split = cfg.data_split if 'data_split' in cfg else cfg.mode
        youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=split)
        if cfg.use_precomp_emb:
            pass
            #global_txt = joblib.load(os.path.join(data_dir,'emb.joblib'))
            #collate_wrapper = MyCollator(txtemb=global_txt)
        #pin_memory=True
        train_loader = DataLoader(youcookdata,\
            batch_size=cfg.batch_size,collate_fn=collate_w,\
                shuffle=True)
           
        net = CrossattnModel(cfg)
        #gpus=1,gpus=3,accelerator='ddp'
        if 'retrain' in cfg and cfg.retrain:
            path = cfg.artifacts_loc+'ckpts/'+cfg.version +'/'
            weight = os.listdir(path)[-1]
            ck_path = path+weight
            print(f"loading from:{ck_path}")
            net = net.load_from_checkpoint(ck_path)
        trainer = pl.Trainer(
            logger=logger_list,callbacks=cbs,deterministic=True,track_grad_norm=2,gradient_clip_val=0.5,**cfg.trainer
        )
        trainer.fit(net, train_loader)
        return trainer
        #trainer.tune(net,train_loader)
            
    else:
        pass
    

In [441]:
path = cfg.artifacts_loc + "ckpts/" + str(cfg.version) + "/"
ckpt = path + os.listdir(path)[0]
ckpt

'/common/home/vk405/Projects/Crossmdl/nbs/ckpts/trail_debug_balanced2/epoch=74-train_loss=0.31.ckpt'

In [442]:
youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=cfg.mode)


In [425]:
# len(youcookdata.data)


In [444]:
net = CrossattnModel(cfg)
net = net.load_from_checkpoint(ckpt)

In [445]:
sample_vid = 'Ysh60eirChU'

ind = youcookdata.global_map[sample_vid]

img = youcookdata.global_imgfeats[ind]

txts = youcookdata.global_txt[sample_vid]
labels = youcookdata.final_labels[sample_vid]

In [428]:
# labels

In [429]:
# trn_l = [ele[-1] for ele in youcookdata.data]


In [430]:
# youcookdata.data

In [431]:
# d = pd.DataFrame({'trn_labels':trn_l})
# for ele in np.linspace(0,1,10):
#     print(f"quantile:{round(ele,2)},value:{d.quantile(ele)[0]}")
    

In [446]:
preds = {}
for start in list(range(0,500-50)):
    s_i = torch.tensor(img[start:start+50])
    s_t  = torch.tensor(txts[1]).unsqueeze(dim=0)
    p = net(([s_i],s_t))
    preds[start] = p.item()



In [433]:
# net = CrossattnModel(cfg)

In [439]:
# for name,param in net.named_parameters():
#     print(name)

In [447]:
#preds

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 1.0,
 12: 1.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 1.0,
 20: 1.0,
 21: 0.9999998807907104,
 22: 1.0,
 23: 1.0,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 0.9999998807907104,
 29: 0.9999998807907104,
 30: 1.0,
 31: 1.0,
 32: 0.9999998807907104,
 33: 0.9999997615814209,
 34: 0.9999998807907104,
 35: 0.9999998807907104,
 36: 0.9999998807907104,
 37: 0.9999997615814209,
 38: 0.9999998807907104,
 39: 0.9999998807907104,
 40: 0.9999998807907104,
 41: 0.9999998807907104,
 42: 0.9999992847442627,
 43: 0.9999992847442627,
 44: 0.9999997615814209,
 45: 1.0,
 46: 1.0,
 47: 1.0,
 48: 1.0,
 49: 1.0,
 50: 1.0,
 51: 1.0,
 52: 1.0,
 53: 1.0,
 54: 1.0,
 55: 1.0,
 56: 1.0,
 57: 1.0,
 58: 1.0,
 59: 1.0,
 60: 1.0,
 61: 1.0,
 62: 1.0,
 63: 1.0,
 64: 1.0,
 65: 1.0,
 66: 1.0,
 67: 1.0,
 68: 1.0,
 69: 1.0,
 70: 1.0,
 71: 1.0,
 72: 1.0,
 73: 1.0,
 74: 1.0,
 75: 1.0,
 76: 1.0,


In [435]:
# sorted(list(preds.items()),key=lambda x:x[-1])[-10:]

In [436]:
# cannot train for smooth interpolation..use some kind of contrastive loss/triplet loss.

In [437]:
from argparse import Namespace
cfg = Namespace(
    version = 'trail_debug_balanced2',
    id = 0,
    artifacts_loc = "/common/home/vk405/Projects/Crossmdl/nbs/",
    data_dir = "/common/home/vk405/Projects/Crossmdl/Data/YouCookII/",
    mode = 'train',
    data_split = 'train',
    loggers = ["csv"],
    seed = 0,
    cbs = ["checkpoint"],
    trainer = {'log_every_n_steps': 50,
    'max_epochs': 150},
    checkpoint = {"every_n_epochs": 1,
    "monitor": "train_loss"},
    use_precomp_emb = True,
    edim = 100,
    attnhdim = 50,
    nheads = 10,
    wrdim = 768,
    vidim = 512,
    hdim = 30,
    dropoutp=0.0,
    batch_size=512,
    framecnt=499,
    retrain=False,
    lr = 5e-4,
    warmup=50,
    max_iters = 2000


)

In [ ]:
run(cfg)

In [ ]:
#.689
import numpy as np




